In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress


In [3]:
import os
from dotenv import load_dotenv

# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
usda_api_key = os.getenv("usda_api_key")
type(usda_api_key)

str

In [4]:
params_url='https://quickstats.nass.usda.gov/api/get_param_values/?key='+usda_api_key
group_desc = requests.get(params_url+'&param=group_desc').json()
commodity_desc=requests.get(params_url+'&param=commodity_desc').json()
region_desc=requests.get(params_url+'&param=region_desc').json()
location_desc=requests.get(params_url+'&param=location_desc').json()
state_alpha=requests.get(params_url+'&param=state_alpha').json()
state_name=requests.get(params_url+'&param=state_name').json()
domaincat_desc=requests.get(params_url+'&param=domaincat_desc').json()
domain_desc=requests.get(params_url+'&param=domain_desc').json()
year=requests.get(params_url+'&param=year').json()
sector_desc=requests.get(params_url+'&param=sector_desc').json()

api_params=[group_desc, commodity_desc,region_desc,domaincat_desc, year]
api_params

[{'group_desc': ['ANIMAL TOTALS',
   'AQUACULTURE',
   'COMMODITIES',
   'CROP TOTALS',
   'DAIRY',
   'ENERGY',
   'EXPENSES',
   'FARMS & LAND & ASSETS',
   'FIELD CROPS',
   'FRUIT & TREE NUTS',
   'HORTICULTURE',
   'INCOME',
   'IRRIGATION',
   'LIVESTOCK',
   'OPERATORS',
   'POULTRY',
   'PRICES PAID',
   'PRODUCERS',
   'SPECIALTY',
   'VEGETABLES']},
 {'commodity_desc': ['AG LAND',
   'AG SERVICES',
   'AG SERVICES & RENT',
   'ALCOHOL COPRODUCTS',
   'ALMONDS',
   'ALPACAS',
   'AMARANTH',
   'ANIMAL PRODUCTS, OTHER',
   'ANIMAL SECTOR',
   'ANIMAL TOTALS',
   'ANIMALS, OTHER',
   'ANNUAL PPI',
   'APPLES',
   'APRICOTS',
   'AQUACULTURE TOTALS',
   'AQUACULTURE, OTHER',
   'AQUATIC PLANTS',
   'ARONIA BERRIES',
   'ARTICHOKES',
   'ASPARAGUS',
   'ASSETS',
   'AUTOMOBILES',
   'AUTOS',
   'AVOCADOS',
   'BAITFISH',
   'BANANAS',
   'BAREROOT HERBACEOUS PERENNIALS',
   'BARLEY',
   'BASIL',
   'BEANS',
   'BEDDING PLANT TOTALS',
   'BEDDING PLANTS, ANNUAL',
   'BEDDING PLANTS

In [205]:
sector_desc

{'sector_desc': ['ANIMALS & PRODUCTS',
  'CROPS',
  'DEMOGRAPHICS',
  'ECONOMICS',
  'ENVIRONMENTAL']}

In [10]:
data_url='https://quickstats.nass.usda.gov/api/api_GET/?key='+usda_api_key 
commodity_list=['CORN','APPLES','PEACHES','BANANAS','APRICOTS','CHERRIES','GRAPES','NECTARINES','LEMONS']
state_list=['CA','IA','NE','TX','IL','KS','MN','IN','NC','WI']

def get_api_results(stateList):
    
    url=data_url+'&source_desc=SURVEY&'+ \
                 '&sector_desc=CROPS&'+ \
                 '&group_desc%3DFRUIT%20TREE%20NUTS'+ \
                 'statisticcat_desc%3DAREA%20OPERATED&'+\
                 'unit_desc=ACRES&'+ \
                 'freq_desc=ANNUAL&'+ \
                 'reference_period_desc=YEAR&'+ \
                 'year__GE=1997&'+ \
                 'agg_level_desc=STATE&'
    
    for state in stateList:
        url=url+'state_alpha='+state+'&'
           
    
    url=url+'format=JSON'
    results_json = requests.get(url).json()
    return results_json



results=get_api_results(state_list)
results_df=pd.DataFrame(results['data'])
results_df.head()
len(results_df),len(results_df[results_df['Value'].str.contains('D')]),len(results_df[results_df['short_desc'].str.contains('ACERS')])


https://quickstats.nass.usda.gov/api/api_GET/?key=48BE989D-D0C4-35F1-91A7-6D301038E6AA&source_desc=SURVEY&&sector_desc=CROPS&&group_desc%3DFRUIT%20TREE%20NUTSstatisticcat_desc%3DAREA%20OPERATED&unit_desc=ACRES&freq_desc=ANNUAL&reference_period_desc=YEAR&year__GE=1997&agg_level_desc=STATE&state_alpha=CA&state_alpha=IA&state_alpha=NE&state_alpha=TX&state_alpha=IL&state_alpha=KS&state_alpha=MN&state_alpha=IN&state_alpha=NC&state_alpha=WI&format=JSON


(18721, 416, 0)

In [11]:
results_df.head()

,end_code,group_desc,domain_desc,state_alpha,state_fips_code,county_ansi,domaincat_desc,watershed_code,commodity_desc,class_desc,...,county_code,reference_period_desc,Value,asd_desc,year,agg_level_desc,prodn_practice_desc,congr_district_code,sector_desc,short_desc
0,00,FIELD CROPS,TOTAL,CA,06,,NOT SPECIFIED,00000000,BARLEY,ALL CLASSES,...,,YEAR,"19,000",,2022,STATE,ALL PRODUCTION PRACTICES,,CROPS,BARLEY - ACRES HARVESTED
1,00,FIELD CROPS,TOTAL,CA,06,,NOT SPECIFIED,00000000,BARLEY,ALL CLASSES,...,,YEAR,"13,000",,2021,STATE,ALL PRODUCTION PRACTICES,,CROPS,BARLEY - ACRES HARVESTED
2,00,FIELD CROPS,TOTAL,CA,06,,NOT SPECIFIED,00000000,BARLEY,ALL CLASSES,...,,YEAR,"33,000",,2020,STATE,ALL PRODUCTION PRACTICES,,CROPS,BARLEY - ACRES HARVESTED
3,00,FIELD CROPS,TOTAL,CA,06,,NOT SPECIFIED,00000000,BARLEY,ALL CLASSES,...,,YEAR,"47,000",,2019,STATE,ALL PRODUCTION PRACTICES,,CROPS,BARLEY - ACRES HARVESTED
4,00,FIELD CROPS,TOTAL,CA,06,,NOT SPECIFIED,00000000,BARLEY,ALL CLASSES,...,,YEAR,"26,000",,2018,STATE,ALL PRODUCTION PRACTICES,,CROPS,BARLEY - ACRES HARVESTED


In [13]:
results_col=results_df.columns
results_col

Index(['end_code', 'group_desc', 'domain_desc', 'state_alpha',
       'state_fips_code', 'county_ansi', 'domaincat_desc', 'watershed_code',
       'commodity_desc', 'class_desc', 'freq_desc', 'week_ending',
       'state_ansi', 'statisticcat_desc', 'watershed_desc', 'source_desc',
       'CV (%)', 'asd_code', 'load_time', 'state_name', 'country_code',
       'zip_5', 'util_practice_desc', 'unit_desc', 'region_desc',
       'country_name', 'county_name', 'location_desc', 'begin_code',
       'county_code', 'reference_period_desc', 'Value', 'asd_desc', 'year',
       'agg_level_desc', 'prodn_practice_desc', 'congr_district_code',
       'sector_desc', 'short_desc'],
      dtype='object')

In [41]:
results_df['class_desc'].unique()

array(['ALL CLASSES', 'DRY EDIBLE, (EXCL CHICKPEAS)', 'DRY EDIBLE, BLACK',
       'DRY EDIBLE, BLACKEYE', 'DRY EDIBLE, CRANBERRY',
       'DRY EDIBLE, DARK, RED, KIDNEY', 'DRY EDIBLE, GREAT NORTHERN',
       'DRY EDIBLE, INCL CHICKPEAS', 'DRY EDIBLE, LIGHT, RED, KIDNEY',
       'DRY EDIBLE, LIMA', 'DRY EDIBLE, LIMA, BABY',
       'DRY EDIBLE, LIMA, LARGE', 'DRY EDIBLE, NAVY', 'DRY EDIBLE, OTHER',
       'DRY EDIBLE, OTHER, NOT LISTED', 'DRY EDIBLE, PINK',
       'DRY EDIBLE, PINTO', 'DRY EDIBLE, RED, KIDNEY',
       'DRY EDIBLE, SMALL, RED', 'DRY EDIBLE, SMALL, WHITE', 'LARGE',
       'SMALL', 'PIMA', 'UPLAND', 'PRINCIPAL, INCL POTATOES',
       '(EXCL ALFALFA)', 'ALFALFA', 'INDUSTRIAL', 'PROSO', 'PEPPERMINT',
       'SPEARMINT', 'DRY EDIBLE', 'LONG GRAIN', 'MEDIUM GRAIN',
       'SHORT GRAIN', 'NON-OIL TYPE', 'OIL TYPE',
       'AIR-CURED LIGHT BURLEY (TYPE 31)',
       'CIGAR BINDER NORTHERN WISCONSIN (TYPE 55)',
       'CIGAR BINDER SOUTHERN WISCONSIN (TYPE 54)',
       'FLUE-CURED 

In [14]:
for col in results_col:
    desc_num=results_df[col].unique()
    num=len(desc_num)
    print (f'Total number of unique fields in {col} :{num}')



Total number of unique fields in end_code :1
Total number of unique fields in group_desc :4
Total number of unique fields in domain_desc :2
Total number of unique fields in state_alpha :10
Total number of unique fields in state_fips_code :10
Total number of unique fields in county_ansi :1
Total number of unique fields in domaincat_desc :2
Total number of unique fields in watershed_code :1
Total number of unique fields in commodity_desc :84
Total number of unique fields in class_desc :86
Total number of unique fields in freq_desc :1
Total number of unique fields in week_ending :1
Total number of unique fields in state_ansi :10
Total number of unique fields in statisticcat_desc :5
Total number of unique fields in watershed_desc :1
Total number of unique fields in source_desc :1
Total number of unique fields in CV (%) :1
Total number of unique fields in asd_code :1
Total number of unique fields in load_time :155
Total number of unique fields in state_name :10
Total number of unique fields

In [20]:
drop_columns=['domain_desc','end_code','state_fips_code',
              'county_ansi','domaincat_desc','watershed_code','freq_desc','week_ending',
              'state_ansi','watershed_desc','watershed_desc', 'source_desc',
       'CV (%)', 'asd_code', 'load_time', 'state_name', 'country_code',
       'zip_5', 'util_practice_desc', 'unit_desc', 'region_desc',
       'country_name', 'county_name', 'location_desc', 'begin_code',
       'county_code', 'reference_period_desc',  'asd_desc', 
       'agg_level_desc', 'prodn_practice_desc', 'congr_district_code',
       'sector_desc']

In [21]:
results_df.drop(columns=drop_columns,axis=1,inplace=True)

In [22]:
results_df.head()

,group_desc,state_alpha,domaincat_desc,commodity_desc,class_desc,statisticcat_desc,Value,year,short_desc
0,FIELD CROPS,CA,NOT SPECIFIED,BARLEY,ALL CLASSES,AREA HARVESTED,"19,000",2022,BARLEY - ACRES HARVESTED
1,FIELD CROPS,CA,NOT SPECIFIED,BARLEY,ALL CLASSES,AREA HARVESTED,"13,000",2021,BARLEY - ACRES HARVESTED
2,FIELD CROPS,CA,NOT SPECIFIED,BARLEY,ALL CLASSES,AREA HARVESTED,"33,000",2020,BARLEY - ACRES HARVESTED
3,FIELD CROPS,CA,NOT SPECIFIED,BARLEY,ALL CLASSES,AREA HARVESTED,"47,000",2019,BARLEY - ACRES HARVESTED
4,FIELD CROPS,CA,NOT SPECIFIED,BARLEY,ALL CLASSES,AREA HARVESTED,"26,000",2018,BARLEY - ACRES HARVESTED


In [42]:
results_df['short_desc'].unique()

array(['BARLEY - ACRES HARVESTED', 'BARLEY - ACRES PLANTED',
       'BARLEY, IRRIGATED - ACRES HARVESTED',
       'BARLEY, IRRIGATED - ACRES PLANTED',
       'BARLEY, NON-IRRIGATED - ACRES HARVESTED',
       'BARLEY, NON-IRRIGATED - ACRES PLANTED',
       'BEANS, DRY EDIBLE, (EXCL CHICKPEAS) - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, (EXCL CHICKPEAS) - ACRES PLANTED',
       'BEANS, DRY EDIBLE, BLACK - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, BLACK - ACRES PLANTED',
       'BEANS, DRY EDIBLE, BLACKEYE - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, BLACKEYE - ACRES PLANTED',
       'BEANS, DRY EDIBLE, CRANBERRY - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, CRANBERRY - ACRES PLANTED',
       'BEANS, DRY EDIBLE, DARK, RED, KIDNEY - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, DARK, RED, KIDNEY - ACRES PLANTED',
       'BEANS, DRY EDIBLE, GREAT NORTHERN - ACRES HARVESTED',
       'BEANS, DRY EDIBLE, GREAT NORTHERN - ACRES PLANTED',
       'BEANS, DRY EDIBLE, GREAT NORTHERN, IRRIGA

In [28]:
results_df['state_alpha'].unique()

array(['CA', 'KS', 'MN', 'NE', 'NC', 'TX', 'WI', 'IL', 'IN', 'IA'],
      dtype=object)

In [30]:
results_df['Value']=results_df['Value'].str.replace(',','')
results_df=results_df[results_df['Value'].str.isdigit()== True]
results_df['Value']=results_df['Value'].astype(int)
results_df['Value'].describe()

count    1.825500e+04
mean     1.190070e+06
std      3.366792e+06
min      0.000000e+00
25%      9.000000e+03
50%      5.000000e+04
75%      4.880000e+05
max      2.502000e+07
Name: Value, dtype: float64

In [35]:
results_df.dtypes

group_desc           object
state_alpha          object
commodity_desc       object
class_desc           object
statisticcat_desc    object
Value                 int32
year                  int64
short_desc           object
dtype: object

In [37]:
results_df.groupby('state_alpha')['Value'].describe()

,count,mean,std,min,25%,50%,75%,max
state_alpha,,,,,,,,
CA,4431.0,1.662507e+05,4.624432e+05,0.0,9000.0,31000.0,114000.0,5193000.0
IA,563.0,4.611349e+06,7.667978e+06,95.0,45000.0,330000.0,9565000.0,25020000.0
IL,996.0,2.464594e+06,5.799435e+06,0.0,9225.0,44350.0,612500.0,23651000.0
IN,894.0,1.445231e+06,3.194133e+06,67.0,4900.0,15000.0,430000.0,12909000.0
KS,1609.0,2.708174e+06,4.600475e+06,0.0,37000.0,545000.0,3388000.0,24421000.0
MN,1786.0,1.260180e+06,3.605768e+06,0.0,19450.0,66000.0,434000.0,20520000.0
NC,1574.0,3.521499e+05,8.705846e+05,160.0,5500.0,18800.0,365000.0,5073000.0
NE,1903.0,1.531209e+06,3.477503e+06,0.0,28000.0,120000.0,1600000.0,19810000.0
TX,3019.0,1.160502e+06,2.979313e+06,0.0,6100.0,38000.0,838000.0,24843000.0


In [38]:
results_df.to_json('products.json')

In [43]:
results_df['year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001,
       2000, 1999, 1998, 1997, 2023], dtype=int64)